<a href="https://colab.research.google.com/github/Samra579/Assignment-04/blob/main/Secure_Data_Encryption_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import hashlib
from cryptography.fernet import Fernet


KEY = Fernet.generate_key()
cipher = Fernet(KEY)


stored_data = {}
failed_attempts = 0


def hash_passkey(passkey):
    return hashlib.sha256(passkey.encode()).hexdigest()


def encrypt_data(text):
    return cipher.encrypt(text.encode()).decode()


def decrypt_data(encrypted_text, passkey):
    global failed_attempts
    hashed_pass = hash_passkey(passkey)

    if encrypted_text in stored_data:
        stored_passkey = stored_data[encrypted_text]["passkey"]
        if hashed_pass == stored_passkey:
            failed_attempts = 0
            decrypted_text = cipher.decrypt(encrypted_text.encode()).decode()
            return decrypted_text
    failed_attempts += 1
    return None



st.set_page_config(page_title="Secure Data System", page_icon="🔒", layout="centered")

st.title("🔒 Secure Data Encryption System")


menu = ["Home", "Store Data", "Retrieve Data", "Login"]
choice = st.sidebar.selectbox("Navigation", menu)


if choice == "Home":
    st.subheader("🏠 Welcome")
    st.write("""
    This app allows you to securely **store** and **retrieve** your data using encryption and passkeys.
    You have **three chances** to provide the correct passkey before reauthorization is required!
    """)

elif choice == "Store Data":
    st.subheader("📂 Store Your Data Securely")

    user_data = st.text_area("Enter the data you want to store:")
    passkey = st.text_input("Set a Passkey (remember it!):", type="password")

    if st.button("Encrypt and Save"):
        if user_data and passkey:
            encrypted_text = encrypt_data(user_data)
            hashed_passkey = hash_passkey(passkey)

            stored_data[encrypted_text] = {"encrypted_text": encrypted_text, "passkey": hashed_passkey}
            st.success("✅ Your data has been securely stored!")
            st.text_area("Here is your Encrypted Text (Save it to retrieve later):", value=encrypted_text, height=100)
        else:
            st.error("⚠️ Please enter both data and passkey!")

elif choice == "Retrieve Data":
    st.subheader("🔍 Retrieve Your Data")

    encrypted_input = st.text_area("Paste your Encrypted Text here:")
    passkey_input = st.text_input("Enter your Passkey:", type="password")

    if st.button("Decrypt"):
        if encrypted_input and passkey_input:
            result = decrypt_data(encrypted_input, passkey_input)

            if result:
                st.success("✅ Successfully Decrypted!")
                st.text_area("Here is your Original Data:", value=result, height=150)
            else:
                attempts_left = 3 - failed_attempts
                st.error(f"❌ Incorrect passkey! Attempts remaining: {attempts_left}")

                if failed_attempts >= 3:
                    st.warning("🔒 Too many failed attempts! Please Reauthorize.")
                    st.session_state["force_login"] = True
                    st.experimental_rerun()
        else:
            st.error("⚠️ Both fields are required!")

elif choice == "Login":
    st.subheader("🔑 Reauthorization Required")

    master_password = st.text_input("Enter Master Password:", type="password")

    if st.button("Login"):
        if master_password == "admin123":
            failed_attempts = 0
            st.success("✅ Reauthorized Successfully!")
            st.experimental_rerun()
        else:
            st.error("❌ Incorrect master password!")

if "force_login" in st.session_state and st.session_state["force_login"]:
    st.warning("🔒 Please Login Again to Continue")
    st.stop()

